In [4]:
import pickle
import os
import shapely
import shapely.geometry
import shapely.affinity
import h5py
import pandas as pd
import tifffile as tiff
from numba import jit, njit
from tqdm import tqdm
from collections import defaultdict
import csv
import sys

import cv2
from shapely.geometry import MultiPolygon, Polygon
import shapely.wkt
import shapely.affinity
import numpy as np
import tifffile as tiff
from shapely.wkt import loads
import collections, numpy

In [5]:
f_label = h5py.File(os.path.join("../data", 'train_label.h5'), 'r')
f_unlabel = h5py.File(os.path.join("../data", 'train_unlabel.h5'), 'r')
f_validate = h5py.File(os.path.join("../data", 'validation.h5'), 'r')

In [6]:
label_mask = np.array(f_label['train_mask'])
unlabel_mask = np.array(f_unlabel['train_mask'])
validate_mask = np.array(f_validate['train_mask'])

In [7]:
def get_class_distribution(mask):
    for i in range(8):
        unique, counts = numpy.unique(mask[:,i,:,:], return_counts=True)
        result = dict(zip(unique, counts))
        if(1 in result):
            portion = result[1]/(result[0]+result[1])
        else:
            portion = 0
        print(portion)

In [8]:
get_class_distribution(label_mask)

0.09907919979659763
0.02145109458787907
0.028039045164571005
0.04330828965768306
0.05299711509569157
0.5110892752923909
0
0.006027149448733358


In [9]:
get_class_distribution(unlabel_mask)

0
0.0016129172059911243
0
0.01345933541743713
0.19002162775344397
0.20205214460925944
0
4.573122284116124e-05


In [10]:
get_class_distribution(validate_mask)

0.020510160006009616
0.011058378501756656
0.008835019444572855
0.045161501190366125
0.060385077662721894
0.795404558351054
0
0


In [13]:
f_test = h5py.File(os.path.join("../data", 'test.h5'), 'r')
test_mask = np.array(f_test['test_mask'])

In [14]:
get_class_distribution(test_mask)

0.01716257129195174
0.005349921051567123
0.005771298380293085
0.02805075560801128
0.05506364269369453
0.22494931192793086
0.00993383283445821
0.0002709563667252219


# Class Distribution of every single image

In [ ]:
def get_all_class_distribution(image,mask):
    index = 0
    result_dict = {}
    for im in image:
        s_image_result = []
        for i in range(8):
            unique, counts = numpy.unique(mask[index,i,:,:], return_counts=True)
            result = dict(zip(unique, counts))
            if(1 in result):
                portion = result[1]/(result[0]+result[1])
            else:
                portion = 0
            s_image_result.append(portion)
        result_dict[im]=s_image_result
        index=index+1
    return result_dict

In [ ]:
image = h5py.File(os.path.join("../data", 'all.h5'), 'r')
mask = np.array(image['image_mask'])

In [ ]:
result_dict = get_all_class_distribution(image['train_ids'],mask)
df = pd.DataFrame(result_dict).T
df.columns = ['Buildings', 'Structure','Road','Track','Trees','Crops','Fast water','Slow water']
df.to_csv('class_distribution.csv')